In [1]:
import pandas
import folium
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import geocoder
import plotly.express as px
import seaborn as sns
import webbrowser
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [2]:
data = pandas.read_csv('sentiment_final.csv')

In [3]:
data = data.set_index('Unnamed: 0')

In [4]:
data

,date,Hiker trail name,Hiker Journal Link,Journal Story,Start location,Destination,Today Miles,Latitude,Longitude,State,Total Shelters,Occurrence,year,month,label,Emotion_scores,state,club,Acronym
Unnamed: 0,,,,,,,,,,,,,,,,,,,
5.0,1/1/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Woke up to 32F. Annie made coffee and we met P...,Hawk Mtn. Shelter\n,gooch mt shelter,7.3,34.655694,-84.049957,Georgia,60.0,4.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.004779126960784...",GA,Georgia Appalachian Trail Club,GATC
6.0,1/1/2016 0:00,Brighty and Wreck,https://www.trailjournals.com//journal/entry/5...,Happy New Year!!! Our next to last section of ...,Ashby Gap\n,trico tower,7.2,44.525200,-70.980900,NaN,29.0,1.0,2016.0,1.0,joy,"[{'label': 'anger', 'score': 0.002314480487257...",ME,Appalachian Mountain Club,AMC
7.0,1/2/2016 0:00,Brother Blood,https://www.trailjournals.com//journal/entry/5...,"It's 2016, and the past two weeks have been mo...",NaN,its 2016,0.0,37.817700,-79.070600,NaN,20.0,1.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.006532554049044...",VA,Tidewater Appalachian Trail Club,TATC
10.0,1/3/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Wow. Yesterday was crazy! I started my period ...,Blood Mtn Shelter\n,mountain crossings hostel,1.5,36.809400,-81.422600,NaN,60.0,1.0,2016.0,1.0,sadness,NaN,VA,Piedmont Appalachian Trail Hikers,PATH
11.0,1/3/2016 0:00,SIMPLE MAN,https://www.trailjournals.com//journal/entry/5...,"Hi,my name is Billy,trail name ""SIMPLE MAN"".I ...",NaN,maine,0.0,40.839400,-75.359800,NaN,0.0,17.0,2016.0,1.0,sadness,"[{'label': 'anger', 'score': 0.002191700274124...",PA,AMC Delaware Valley Chapter,AMC-DV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33192.0,10/12/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/4...,Our 2014 goal of getting to Swatara Gap is com...,Route 183\n,rt 501,9.0,40.512510,-76.346900,NaN,1.0,1.0,2023.0,10.0,neutral,"[{'label': 'anger', 'score': 0.012704795226454...",PA,Blue Mountain Eagle Climbing Club,BMECC
33212.0,11/9/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/6...,Nick had a rough night in hotel. I guess he w...,Neighbor Mountain Parking\n,hogback,13.0,35.963950,-82.587200,NaN,1.0,1.0,2023.0,11.0,fear,"[{'label': 'anger', 'score': 0.009218733757734...",NC,Carolina Mountain Club,CMC
33224.0,12/5/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/5...,It was Tyler's Birthday and he wanted money to...,Route 225 Pedestrian Bridge Parking Lot\n,clarks ferry bridge,6.7,40.392220,-76.994000,NaN,1.0,1.0,2023.0,12.0,neutral,"[{'label': 'anger', 'score': 0.014029124751687...",PA,NaN,NaN


## Folium

In [5]:
# t = data[data['year'] == 2016]
# t = t[t['label'] == 'joy']

In [6]:
# #color = ['blue', 'orange', 'purple', 'green', 'pink', 'cyan', 'lime', 'yellow', 'red', 'navy', 'brown']

# #pin_color = ['darkblue', 'orange', 'purple', 'green', 'pink', 'lightblue', 'lightgreen', 'beige', 'red']
# map = folium.Map(location=[38, -84], zoom_start=6, min_zoom=5, max_bounds=True)
# for index in range(len(t)):
#     df = t.iloc[index]
#     #popup = folium.Popup(htmldiplay(data_clean[data_clean['beachid'] == index]), max_width=900)
#     marker = folium.Marker([df['Latitude'], df['Longitude']])
#     marker.add_to(map)

# map

## Plotly: Emotion Distribution over States

In [7]:
emotion = 'neutral'
year = 2020
month = 7

In [8]:
df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]['state'].value_counts().reset_index().sort_values('state')
df

,state,count
8,CT,1
4,GA,2
3,MA,5
1,NC,6
7,NY,1
6,PA,1
5,TN,2
0,VA,8
2,VT,5


In [9]:
a = df['count'].reset_index(drop=True)
#data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]['state'].value_counts().reindex(data.state.unique(), fill_value=0).sort_index()
a

0    1
1    2
2    5
3    6
4    1
5    1
6    2
7    8
8    5
Name: count, dtype: int64

In [10]:
b = data[(data['year'] == year) & (data['month'] == month)]['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
b

0     4
1     5
2     7
3    13
4     2
5     6
6     7
7    22
8     8
Name: count, dtype: int64

In [32]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion distribution over States'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2023,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value")]
)
def display_choropleth(emotion, year, month, month_toggle):
    if 'ignore' in month_toggle:
        df = data[(data['label'] == emotion) & (data['year'] == year)]['state'].value_counts().reset_index().sort_values('state')
        b = data[(data['year'] == year)]['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
    else:
        df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]['state'].value_counts().reset_index().sort_values('state')
        b = data[(data['year'] == year) & (data['month'] == month)]['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
    a = df['count'].reset_index(drop=True)
    df['proportion'] = round(a/b, 4)*100
    
    
    fig = px.choropleth(
        df,
        locationmode="USA-states",
        locations='state',
        color='proportion',
        scope="usa",
        color_continuous_scale="OrRd",
        range_color=(0,100),
        title='Emotion distribution over States'
    )
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig


webbrowser.open('http://127.0.0.1:8050/', new=2, autoraise=True)
app.run_server(debug=True)

## Ploty: Emotion Distribution over Clubs

In [12]:
# g = pandas.read_excel("Trail Clubs' Geo-location data.xlsx")

In [13]:
# def get_label_for_coordinate(lat, lon, df):
#     """
#     Get the label for a coordinate if it falls into any box defined in the dataframe.

#     Parameters:
#     - lat (float): Latitude of the point.
#     - lon (float): Longitude of the point.
#     - df (pd.DataFrame): Dataframe containing the box coordinates and labels.

#     Returns:
#     - str: Label of the box the point falls into or None if the point is outside all boxes.
#     """
#     if pandas.isna(lat) or pandas.isna(lon):
#         return pandas.NA
    
#     try:
#         lat = float(lat)
#         lon = float(lon)
#     except:
#         return pandas.NA
#     for index, row in df.iterrows():
#         if (row['latitude_start'] <= lat <= row['latitude_end']) and (row['longitude_start'] <= lon <= row['longitude_end']):
#             return row['Acronym']
    
#     return pandas.NA

In [14]:
# p = pandas.read_excel('state_added_update.xlsx')

In [15]:
# p

In [16]:
# p['Acronym'] = p.apply(lambda row: get_label_for_coordinate(row['Latitude'], row['Longitude'], g), axis=1)

In [17]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion distribution over Clubs'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2023,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value")],
     Input("month-toggle", "value")
)
def display_choropleth(emotion, year, month, month_toggle):
    if 'ignore' in month_toggle:
        df = data[(data['label'] == emotion) & (data['year'] == year)]
    else:
        df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]

    # Use seaborn to generate a palette of 24 colors
    colors = sns.color_palette("husl", 24).as_hex()

    # Create a color map for each unique club
    clubs = df['club'].unique()
    if len(clubs) > len(colors):
        raise ValueError("The number of unique clubs exceeds available colors.")

    color_map = dict(zip(clubs, colors))
    
    # Create an empty figure
    fig = go.Figure()

    # Add a trace for each club
    for club in clubs:
        subset = df[df['club'] == club]
        fig.add_trace(go.Scattergeo(
            lon = subset['Longitude'],
            lat = subset['Latitude'],
            text = subset['label'],
            mode = 'markers',
            marker = dict(
                color = color_map[club],
                size = 8,
                opacity = 0.3
            ),
            name = club,
            showlegend=True
        ))

    fig.update_layout(
            # title = 'Emotion distribution over Clubs',
            geo_scope='usa',
        )

    return fig


app.run_server(debug=True)

## Bar plot for Propoertion 1

In [18]:
data['state']

Unnamed: 0
5.0        GA
6.0        ME
7.0        VA
10.0       VA
11.0       PA
           ..
33192.0    PA
33212.0    NC
33224.0    PA
33233.0    GA
33253.0    PA
Name: state, Length: 15489, dtype: object

In [19]:
data

,date,Hiker trail name,Hiker Journal Link,Journal Story,Start location,Destination,Today Miles,Latitude,Longitude,State,Total Shelters,Occurrence,year,month,label,Emotion_scores,state,club,Acronym
Unnamed: 0,,,,,,,,,,,,,,,,,,,
5.0,1/1/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Woke up to 32F. Annie made coffee and we met P...,Hawk Mtn. Shelter\n,gooch mt shelter,7.3,34.655694,-84.049957,Georgia,60.0,4.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.004779126960784...",GA,Georgia Appalachian Trail Club,GATC
6.0,1/1/2016 0:00,Brighty and Wreck,https://www.trailjournals.com//journal/entry/5...,Happy New Year!!! Our next to last section of ...,Ashby Gap\n,trico tower,7.2,44.525200,-70.980900,NaN,29.0,1.0,2016.0,1.0,joy,"[{'label': 'anger', 'score': 0.002314480487257...",ME,Appalachian Mountain Club,AMC
7.0,1/2/2016 0:00,Brother Blood,https://www.trailjournals.com//journal/entry/5...,"It's 2016, and the past two weeks have been mo...",NaN,its 2016,0.0,37.817700,-79.070600,NaN,20.0,1.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.006532554049044...",VA,Tidewater Appalachian Trail Club,TATC
10.0,1/3/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Wow. Yesterday was crazy! I started my period ...,Blood Mtn Shelter\n,mountain crossings hostel,1.5,36.809400,-81.422600,NaN,60.0,1.0,2016.0,1.0,sadness,NaN,VA,Piedmont Appalachian Trail Hikers,PATH
11.0,1/3/2016 0:00,SIMPLE MAN,https://www.trailjournals.com//journal/entry/5...,"Hi,my name is Billy,trail name ""SIMPLE MAN"".I ...",NaN,maine,0.0,40.839400,-75.359800,NaN,0.0,17.0,2016.0,1.0,sadness,"[{'label': 'anger', 'score': 0.002191700274124...",PA,AMC Delaware Valley Chapter,AMC-DV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33192.0,10/12/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/4...,Our 2014 goal of getting to Swatara Gap is com...,Route 183\n,rt 501,9.0,40.512510,-76.346900,NaN,1.0,1.0,2023.0,10.0,neutral,"[{'label': 'anger', 'score': 0.012704795226454...",PA,Blue Mountain Eagle Climbing Club,BMECC
33212.0,11/9/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/6...,Nick had a rough night in hotel. I guess he w...,Neighbor Mountain Parking\n,hogback,13.0,35.963950,-82.587200,NaN,1.0,1.0,2023.0,11.0,fear,"[{'label': 'anger', 'score': 0.009218733757734...",NC,Carolina Mountain Club,CMC
33224.0,12/5/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/5...,It was Tyler's Birthday and he wanted money to...,Route 225 Pedestrian Bridge Parking Lot\n,clarks ferry bridge,6.7,40.392220,-76.994000,NaN,1.0,1.0,2023.0,12.0,neutral,"[{'label': 'anger', 'score': 0.014029124751687...",PA,NaN,NaN


In [20]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion proportion over States'),

    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    html.P("All Time:"),
    dcc.Checklist(
        id='all-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value"),
     Input("all-toggle", "value")]
)
def display_choropleth(year, month, month_toggle, all_toggle):
    if 'ignore' in all_toggle:
        df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    else:
        if 'ignore' in month_toggle:
            df = data[(data['year'] == year)].groupby(['state', 'label']).size().unstack(fill_value=0)
        else:
            df = data[(data['year'] == year) & (data['month'] == month)].groupby(['state', 'label']).size().unstack(fill_value=0)
    #df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    row_sums = df.sum(axis=1)
    proportions_df = df.div(row_sums, axis=0)
    proportions_df = proportions_df.apply(lambda x: round(x*100, 2))
    fig = px.bar(proportions_df,
                barmode='stack',
                labels={'index': 'X-axis Labels', 'value': 'Proportion distribution in percentage(%)'},
                title='Stacked Bar Plot')
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig
app.run_server(debug=True)

## Bar plot for Propoertion 2

In [21]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion proportion over clubs'),

    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    html.P("All Time:"),
    dcc.Checklist(
        id='all-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value"),
     Input("all-toggle", "value")]
)
def display_choropleth(year, month, month_toggle, all_toggle):
    if 'ignore' in all_toggle:
        df = data.groupby(['club', 'label']).size().unstack(fill_value=0)
    else:
        if 'ignore' in month_toggle:
            df = data[(data['year'] == year)].groupby(['club', 'label']).size().unstack(fill_value=0)
        else:
            df = data[(data['year'] == year) & (data['month'] == month)].groupby(['club', 'label']).size().unstack(fill_value=0)
    #df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    row_sums = df.sum(axis=1)
    proportions_df = df.div(row_sums, axis=0)
    proportions_df = proportions_df.apply(lambda x: round(x*100, 2))
    fig = px.bar(proportions_df,
                barmode='stack',
                labels={'index': 'X-axis Labels', 'value': 'Proportion distribution in percentage(%)'},
                title='Stacked Bar Plot')
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

app.run_server(debug=True)

## Integration

In [33]:
app = Dash(__name__, suppress_callback_exceptions=True)

########################################################################################################################################################################################################################################################################################
# 1
tab1_layout = html.Div([
    html.H4('Emotion distribution over States'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    html.P("All Time:"),
    dcc.Checklist(
        id='all-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph1"),
])

@app.callback(
    Output("graph1", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value"),
     Input("all-toggle", "value")]
)
def callback1(emotion, year, month, month_toggle, all_toggle):
    if 'ignore' in all_toggle:
        df = data[data['label'] == emotion]['state'].value_counts().reset_index().sort_values('state')
        b = data['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
    else:
        if 'ignore' in month_toggle:
            df = data[(data['label'] == emotion) & (data['year'] == year)]['state'].value_counts().reset_index().sort_values('state')
            b = data[(data['year'] == year)]['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
        else:
            df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]['state'].value_counts().reset_index().sort_values('state')
            b = data[(data['year'] == year) & (data['month'] == month)]['state'].value_counts().reindex(df.state.unique(), fill_value=0).sort_index().reset_index(drop=True)
    a = df['count'].reset_index(drop=True)
    df['proportion'] = round(a/b, 4)*100
    
    
    fig = px.choropleth(
        df,
        locationmode="USA-states",
        locations='state',
        color='proportion',
        scope="usa",
        color_continuous_scale="OrRd",
        #range_color=(0,100),
        title='Emotion distribution over States'
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig


########################################################################################################################################################################################################################################################################################
# 2
tab2_layout = html.Div([
    html.H4('Emotion distribution over Clubs'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph2"),
])

@app.callback(
    Output("graph2", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value")]
)
def callback2(emotion, year, month, month_toggle):
    if 'ignore' in month_toggle:
        df = data[(data['label'] == emotion) & (data['year'] == year)]
    else:
        df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]

    # Use seaborn to generate a palette of 24 colors
    colors = sns.color_palette("husl", 24).as_hex()

    # Create a color map for each unique club
    clubs = df['club'].unique()
    if len(clubs) > len(colors):
        raise ValueError("The number of unique clubs exceeds available colors.")

    color_map = dict(zip(clubs, colors))
    
    # Create an empty figure
    fig = go.Figure()

    # Add a trace for each club
    for club in clubs:
        subset = df[df['club'] == club]
        fig.add_trace(go.Scattergeo(
            lon = subset['Longitude'],
            lat = subset['Latitude'],
            text = subset['label'],
            mode = 'markers',
            marker = dict(
                color = color_map[club],
                size = 8,
                opacity = 0.3
            ),
            name = club,
            showlegend=True
        ))

    fig.update_layout(
            #title = 'Emotion distribution over Clubs',
            geo_scope='usa',
        )

    return fig

########################################################################################################################################################################################################################################################################################
# 3
tab3_layout = html.Div([
    html.H4('Emotion proportion over States'),

    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    html.P("All Time:"),
    dcc.Checklist(
        id='all-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph3"),
])

@app.callback(
    Output("graph3", "figure"), 
    [Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value"),
     Input("all-toggle", "value")]
)
def callback3(year, month, month_toggle, all_toggle):
    if 'ignore' in all_toggle:
        df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    else:
        if 'ignore' in month_toggle:
            df = data[(data['year'] == year)].groupby(['state', 'label']).size().unstack(fill_value=0)
        else:
            df = data[(data['year'] == year) & (data['month'] == month)].groupby(['state', 'label']).size().unstack(fill_value=0)
    #df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    row_sums = df.sum(axis=1)
    proportions_df = df.div(row_sums, axis=0)
    proportions_df = proportions_df.apply(lambda x: round(x*100, 2))
    order = ['TN', 'NC', 'GA', 'VA', 'MD', 'NJ', 'PA', 'NY', 'CT', 'MA', 'NH', 'VT', 'ME']
    proportions_df = proportions_df.reindex(order)
    fig = px.bar(proportions_df,
                barmode='stack',
                labels={'index': 'X-axis Labels', 'value': 'Proportion distribution in percentage(%)'},
                title='Stacked Bar Plot')
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig
########################################################################################################################################################################################################################################################################################
# 4
tab4_layout = html.Div([
    html.H4('Emotion proportion over clubs'),

    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2016,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    html.P("Whole Year:"),
    dcc.Checklist(
        id='month-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    html.P("All Time:"),
    dcc.Checklist(
        id='all-toggle',
        options=[{'label': '', 'value': 'ignore'}],
        value=[]
    ),
    dcc.Graph(id="graph4"),
])

@app.callback(
    Output("graph4", "figure"), 
    [Input("year-slider", "value"),
     Input("month-slider", "value"),
     Input("month-toggle", "value"),
     Input("all-toggle", "value")]
)
def callback4(year, month, month_toggle, all_toggle):
    if 'ignore' in all_toggle:
        df = data.groupby(['club', 'label']).size().unstack(fill_value=0)
    else:
        if 'ignore' in month_toggle:
            df = data[(data['year'] == year)].groupby(['club', 'label']).size().unstack(fill_value=0)
        else:
            df = data[(data['year'] == year) & (data['month'] == month)].groupby(['club', 'label']).size().unstack(fill_value=0)
    #df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
    row_sums = df.sum(axis=1)
    proportions_df = df.div(row_sums, axis=0)
    proportions_df = proportions_df.apply(lambda x: round(x*100, 2))
    fig = px.bar(proportions_df,
                barmode='stack',
                labels={'index': 'X-axis Labels', 'value': 'Proportion distribution in percentage(%)'},
                title='Stacked Bar Plot')
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig
########################################################################################################################################################################################################################################################################################
# Main Manu

app.layout = html.Div([
    dcc.Tabs(id='tabs', value='tab1', children=[
        dcc.Tab(label='Emotion distribution by State', value='tab1'),
        dcc.Tab(label='Emotion distribution by Club', value='tab2'),
        dcc.Tab(label='Emotion proportion by State', value='tab3'),
        dcc.Tab(label='Emotion proportion by Clubs', value='tab4')
    ]),
    html.Div(id='tabs-content')
])

@app.callback(Output('tabs-content', 'children'), [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab1':
        return tab1_layout
    elif tab == 'tab2':
        return tab2_layout
    elif tab == 'tab3':
        return tab3_layout
    elif tab == 'tab4':
        return tab4_layout

app.run_server(debug=True)


In [26]:

df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
#df = data.groupby(['state', 'label']).size().unstack(fill_value=0)
row_sums = df.sum(axis=1)
proportions_df = df.div(row_sums, axis=0)
proportions_df = proportions_df.apply(lambda x: round(x*100, 2))
fig = px.bar(proportions_df,
        barmode='stack',
        labels={'index': 'X-axis Labels', 'value': 'Proportion distribution in percentage(%)'},
        title='Stacked Bar Plot')
#fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [30]:
proportions_df.index

Index(['CT', 'GA', 'MA', 'MD', 'ME', 'NC', 'NH', 'NJ', 'NY', 'PA', 'TN', 'VA',
       'VT'],
      dtype='object', name='state')